# #1 K_Means (DTW) with Tslearn

Aplicaremos el algoritmo K-Means utilizando la librería tslearn, con una métrica basada en DTW (Dynamic Time Warping). Esta combinación permite comparar series temporales que no están perfectamente alineadas en el tiempo (por ejemplo, viviendas cuyos consumos pueden estar desfasados o tener hábitos distintos de horario), y agruparlas en clústeres con patrones de consumo similares.

La métrica DTW es especialmente útil en este caso porque:

Permite comparar series temporales de distinta forma o fase, alineando picos y valles aunque ocurran en momentos diferentes.

No requiere que las viviendas tengan datos para exactamente las mismas fechas u horas.

Es robusta frente a pequeñas diferencias en el tiempo (por ejemplo, si dos viviendas consumen más por la noche, pero una empieza a las 22h y otra a las 23h).

In [5]:
%pip install pandas numpy tslearn scikit-learn matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\tipir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
from tslearn.datasets import CachedDatasets
from sklearn.base import BaseEstimator, TransformerMixin
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler
from sklearn.metrics import silhouette_score
from tslearn.clustering import TimeSeriesKMeans
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/vertical_preprocessed_data.csv", sep=";")

import warnings
warnings.filterwarnings("ignore")

## Definicion de las clases

Primero definimos las clases que van a formar nuestro pipeline, son 3 etapas: Preprocesamiento, seleccion del numero de cluster,ejecucion del clustering y valoracion de resultados.

In [7]:
class ClusterSelection:
    def __init__(self, max_clusters=10, metric='dtw'):
        self.max_clusters = max_clusters
        self.metric = metric

    def select_optimal_clusters(self, X):
        inertia = []  # Para el método del codo
        silhouette_scores = []  # Para la puntuación de Silhouette

        for k in range(2, self.max_clusters + 1):
            # Crear el modelo de KMeans para cada valor de k
            model = TimeSeriesKMeans(n_clusters=k, metric=self.metric, random_state=42)
            labels = model.fit_predict(X)
            
            # Calcular la inercia (distorsión)
            inertia.append(model.inertia_)
            
            # Calcular el Silhouette Score
            silhouette = silhouette_score(X, labels)
            silhouette_scores.append(silhouette)
        
        # Graficar los resultados
        plt.figure(figsize=(10, 5))
        
        # Método del Codo
        plt.subplot(1, 2, 1)
        plt.plot(range(2, self.max_clusters + 1), inertia, marker='o')
        plt.title("Método del Codo")
        plt.xlabel("Número de Clústeres")
        plt.ylabel("Inercia")
        
        # Silhouette Score
        plt.subplot(1, 2, 2)
        plt.plot(range(2, self.max_clusters + 1), silhouette_scores, marker='o', color='orange')
        plt.title("Silhouette Score")
        plt.xlabel("Número de Clústeres")
        plt.ylabel("Silhouette Score")
        
        plt.tight_layout()
        plt.show()
        
        # Retornar el mejor número de clústeres según el Silhouette Score
        optimal_clusters = range(2, self.max_clusters + 1)[np.argmax(silhouette_scores)]
        print(f"El número óptimo de clústeres basado en el Silhouette Score es: {optimal_clusters}")
        
        return optimal_clusters


In [8]:
class TimeSeriesPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Ordenar los datos por 'cups' y 'fecha_hora_str' (ya que están en formato largo)
        X_sorted = X.sort_values(by=['cups', 'fecha', 'hora'])

        # Crear una lista para almacenar las series temporales
        series_list = []
        for cup, group in X_sorted.groupby('cups'):
            # Crear una serie de consumo para cada 'cups' (con datos de cada hora)
            series = group['consumo_kWh'].values
            series_list.append(series)
        
        # Convertimos a un array de numpy
        X_np = np.array(series_list)

        # Normalizar las series (si es necesario)
        scaler = TimeSeriesScalerMeanVariance()
        X_scaled = scaler.fit_transform(X_np)
        
        return X_scaled

In [9]:
class TimeSeriesClustering(BaseEstimator):
    def __init__(self, n_clusters=None, metric='dtw'):
        self.n_clusters = n_clusters
        self.metric = metric
        self.cluster_selector = ClusterSelection(max_clusters=10, metric=self.metric)

    def fit(self, X, y=None):
        # Si no se especifica el número de clústeres, lo seleccionamos automáticamente
        if self.n_clusters is None:
            self.n_clusters = self.cluster_selector.select_optimal_clusters(X)
        
        # Ajustamos el modelo de clustering
        self.model = TimeSeriesKMeans(n_clusters=self.n_clusters, metric=self.metric, random_state=42)
        self.labels_ = self.model.fit_predict(X)
        return self

    def transform(self, X):
        return self.labels_

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

In [10]:
# Paso 3: Visualización y chequeo de resultados
class CheckResults(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, labels):
        # Visualizar la distribución de los clústeres
        sns.countplot(x=labels)
        plt.title("Distribución de viviendas por clúster")
        plt.xlabel("Cluster")
        plt.ylabel("Número de viviendas")
        plt.show()
        return labels


In [11]:
# Ejecutamos el pipeline con tu DataFrame df (en formato largo)
pipeline = Pipeline([
    ('preprocessor', TimeSeriesPreprocessor()),
    ('clustering', TimeSeriesClustering(n_clusters=3, metric="dtw")),
    ('results', CheckResults())
])

# Ejecutar el pipeline con tu DataFrame df (que ya está en formato largo)
pipeline.fit_transform(df)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (131,) + inhomogeneous part.